In [2]:
import data_engine as de
import pandas as pd
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer,CategoricalImputer

In [3]:
data_save_path = './data/'
root_path = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD\\'
pattern = '*BANKNIFTY.txt'
data_name = 'banknifty'
#base_df= de.create_dataset(root_path,pattern,data_save_path,data_name)
base_df = pd.read_csv(r'C:\Users\8prab\Google Drive\Work\trading\data\banknifty\banknifty.csv')

In [4]:
%%time
from sklearn.pipeline import Pipeline
shft_col_pat = ['close','open','high','low','momentum','volatile','transform','pattern','overlap']
shft_col_pat = ['close','open']

pipe = Pipeline([
     ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True)),
     ('nd3', de.NormalizeDataset(column_pattern = ['close','open','high','low','momentum','volatile','transform','pattern','overlap'],drop_na_rows=False,fillna=True,fillna_method='ffill')),
   # ('pdrhw', de.PriceDayRangeHourWise(first_col = 'high',second_col='low',hour_range = [('09:00', '10:30'),('10:30', '11:30')],range_type=['price_range','price_deviation_max_first_col'])),
    #('pv', de.PricePerIncrement(freq='1min',shift=3,shift_column=[],shift_column_pattern=shft_col_pat,#verbose=False)),
    ('pltbc', de.PriceLastTickBreachCountv1(column_pattern=shft_col_pat,columns=[],last_ticks='10min',breach_type = ['morethan','max'])),
    #('pltbc2', de.PriceLastTickBreachCount(column_pattern=shft_col_pat,columns=[],create_new_col = True,last_ticks='10min',breach_type = ['morethan','max'])),
   # ('ppi', de.PricePerIncrement(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ])
base_df2 = pipe.fit(base_df).transform(base_df)


****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
PriceLastTickBreachCount : morethan
PriceLastTickBreachCount : max
Shape of dataframe after PriceLastTickBreachCount is (1105219, 8)
Wall time: 16min 17s


In [4]:
base_df1

,open,high,low,close,last_tick_morethan_open_10min,last_tick_max_open_10min,last_tick_morethan_close_10min,last_tick_max_close_10min
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,0.0,0,0.0,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,1.0,0,1.0,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,2.0,0,1.0,0
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,2.0,0,2.0,0
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,3.0,0,1.0,0
...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,2.0,0,4.0,0
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,6.0,0,0.0,0
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,2.0,0,2.0,0
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,3.0,0,3.0,0


In [5]:
base_df2

,open,high,low,close,close_10min_morethan_max_last_tick__x,open_10min_morethan_max_last_tick__x,close_10min_morethan_max_last_tick__y,open_10min_morethan_max_last_tick__y
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,0.0,0.0,0,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,1.0,1.0,0,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,1.0,2.0,0,0
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,2.0,2.0,0,0
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,1.0,3.0,0,0
...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,4.0,2.0,0,0
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,0.0,6.0,0,0
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,2.0,2.0,0,0
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,3.0,3.0,0,0


In [31]:
import numpy as np

tes1 = base_df1.between_time('09:00', '10:00')
#tes1['close'].rolling('10min', min_periods=1).apply(lambda x:sum(np.array(x[:-1]) > np.mean(np.array(x))))
tes1['close'].rolling('10min', min_periods=1).apply(lambda x:sum(x > np.mean(np.array(x))))

#t = np.array([2343,234,245,1124]) > 444
#sum(t)

2010-03-02 09:01:00    0.0
2010-03-02 09:02:00    1.0
2010-03-02 09:03:00    2.0
2010-03-02 09:04:00    3.0
2010-03-02 09:05:00    3.0
                      ... 
2022-01-13 09:56:00    5.0
2022-01-13 09:57:00    4.0
2022-01-13 09:58:00    6.0
2022-01-13 09:59:00    6.0
2022-01-13 10:00:00    7.0
Name: close, Length: 135805, dtype: float64

In [29]:
tes1['close'].rolling('10min', min_periods=1).apply(lambda x:( x[:-1] > x[-1]).sum())

2010-03-02 09:01:00    0.0
2010-03-02 09:02:00    0.0
2010-03-02 09:03:00    1.0
2010-03-02 09:04:00    1.0
2010-03-02 09:05:00    3.0
                      ... 
2022-01-13 09:56:00    7.0
2022-01-13 09:57:00    9.0
2022-01-13 09:58:00    9.0
2022-01-13 09:59:00    8.0
2022-01-13 10:00:00    7.0
Name: close, Length: 135805, dtype: float64

In [36]:
last_ticks1='10min'
last_ticks2='60min'
breach_type = ['morethan','max']
c = 'mean'
c1 = 'mean'
cols = ['open','high','low']
oper = ['-','+','/','+']
aggs = ['mean','sum','max','median']
last_ticks = ['2min','3min','5min','20min']
eval_stmt = ''
if len(oper) == len(aggs):
    oper.append('dummy')

for lt,oper,agg in zip(last_ticks,oper,aggs):
    print(lt,oper,agg)
    tmpst = f"base_df1[{cols}].rolling('{lt}', min_periods=1).{agg}() {oper}"
    eval_stmt = eval_stmt + tmpst
#base_df1[['open','high']].rolling(last_ticks1, min_periods=1).mean() - base_df1[['open','high']].rolling(last_ticks2, min_periods=1).mean()
#l = f"base_df1[{col}].rolling(last_ticks1, min_periods=1).{c}() - base_df1[{col}].rolling(last_ticks2, min_periods=1).{c1}()"
#l2 = eval(l)
#l2
print(eval_stmt[:-1])
eval(eval_stmt[:-1])

2min - mean
3min + sum
5min / max
20min + median
base_df1[['open', 'high', 'low']].rolling('2min', min_periods=1).mean() -base_df1[['open', 'high', 'low']].rolling('3min', min_periods=1).sum() +base_df1[['open', 'high', 'low']].rolling('5min', min_periods=1).max() /base_df1[['open', 'high', 'low']].rolling('20min', min_periods=1).median() 


,open,high,low
2010-03-02 09:01:00,1.000000,1.000000,1.000000
2010-03-02 09:02:00,-8832.824216,-8854.424828,-8828.849666
2010-03-02 09:03:00,-17674.123309,-17710.749859,-17664.623658
2010-03-02 09:04:00,-17689.523485,-17706.099757,-17675.999279
2010-03-02 09:05:00,-17699.823660,-17703.249656,-17684.399219
...,...,...,...
2022-01-13 15:28:00,-76932.749970,-76941.874993,-76921.349858
2022-01-13 15:29:00,-76931.925006,-76941.299854,-76916.699918
2022-01-13 15:30:00,-76929.875006,-76943.849854,-76914.349918
2022-01-13 15:31:00,-76933.049993,-76947.649854,-76916.249843


In [38]:
a = ['close','open']
'_'.join(a)

'close_open'

In [25]:
base_df1

,open,high,low,close,close_1min_3_price_per_velocity,open_1min_3_price_per_velocity
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,NaN,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,NaN,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,NaN,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,2.300000,5.650000
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,-3.566667,1.883333
...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,-1.300000,-0.566667
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,-3.650000,0.183333
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,0.166667,-0.733333
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,-0.383333,0.566667


In [10]:
base_df[['ihgh','peno']] = base_df[['open','high']]
print(base_df)
base_df[['open','high']] = base_df[['open','high']]-base_df[['ihgh','peno']]
base_df

                     open  high       low     close         x         y  xxxx  \
2018-04-02 09:08:00   0.0   0.0  24235.70  24235.70  24235.70  24235.70   0.0   
2018-04-02 09:16:00   0.0   0.0  24243.70  24253.10  24268.00  24300.60   0.0   
2018-04-02 09:17:00   0.0   0.0  24219.30  24221.40  24257.40  24286.10   0.0   
2018-04-02 09:18:00   0.0   0.0  24204.00  24220.70  24222.60  24224.40   0.0   
2018-04-02 09:19:00   0.0   0.0  24218.70  24229.40  24218.70  24247.10   0.0   
...                   ...   ...       ...       ...       ...       ...   ...   
2011-12-30 15:26:00   0.0   0.0   7954.30   7956.25   7955.80   7956.25   0.0   
2011-12-30 15:27:00   0.0   0.0   7955.80   7959.80   7955.80   7959.90   0.0   
2011-12-30 15:28:00   0.0   0.0   7960.90   7961.40   7960.90   7963.45   0.0   
2011-12-30 15:29:00   0.0   0.0   7953.30   7954.70   7959.45   7959.45   0.0   
2011-12-30 15:30:00   0.0   0.0   7952.55   7954.75   7955.30   7956.90   0.0   

                     yyyy  

ValueError: Columns must be same length as key

In [5]:
%%time
from sklearn.pipeline import Pipeline
pipe = Pipeline([
     ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True)),
   # ('pdrhw', de.PriceDayRangeHourWise(first_col = 'high',second_col='low',hour_range = [('09:00', '10:30'),('10:30', '11:30')],range_type=['price_range','price_deviation_max_first_col'])),
    ('pv', de.PriceVelocity(freq='1min',shift=3,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
   # ('ppi', de.PricePerIncrement(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ])
base_df2 = pipe.fit(base_df).transform(base_df)


****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
PriceVelocity : low
PriceVelocity : high
PriceVelocity : open
PriceVelocity : close
Wall time: 3.88 s


In [6]:
base_df2

,open,high,low,close,price_velocity_low_1min_3,price_velocity_high_1min_3,price_velocity_open_1min_3,price_velocity_close_1min_3
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,NaN,NaN,NaN,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,NaN,NaN,NaN,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,NaN,NaN,NaN,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,16.85,-6.25,16.95,6.90
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,4.95,-7.50,5.65,-10.70
...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,-0.95,-3.05,-1.70,-3.90
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,-6.15,5.45,0.55,-10.95
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,-4.50,3.75,-2.20,0.50
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,6.60,-3.10,1.70,-1.15


In [7]:
from sklearn.pipeline import Pipeline
pipe1 = Pipeline([
    ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True)),
    #('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),
    #('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),
    ('tech_indicator', de.TechnicalIndicator(method_type = ['momentum','volatile','transform','pattern','overlap'])),
    ('nd2', de.NormalizeDataset(column_pattern = ['close','open','high','low','momentum','volatile','transform','pattern','overlap'],fillna=True,fillna_method='bfill')),
    ('nd3', de.NormalizeDataset(column_pattern = ['close','open','high','low','momentum','volatile','transform','pattern','overlap'],drop_na_rows=False,fillna=True,fillna_method='ffill')),
    ('ltgvc', de.LastTicksGreaterValuesCount(column_pattern=['close','open','momentum','volatile','transform','pattern','overlap'])),
    #('nd3', de.NormalizeDataset(columns = ['close','open','high','low'],fillna=True)),
    ])
base_df = pipe1.fit(base_df).transform(base_df)

****************************************************************************************************
Shape of dataframe after NormalizeDataset is (1105219, 4)
****************************************************************************************************
Function momentum_AROON was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_AROONOSC was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_MFI was unable to run, Error is 'Signals' object has no attribute 'volume'
Function pattern_abondoned_baby was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_dark_cloud_cover was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_doji_star was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_star was unable to run, Error is I

KeyboardInterrupt: 

In [4]:
pipe2 = Pipeline([
    ('pdrhw', de.PriceDayRangeHourWise(first_col = 'high',second_col='low',hour_range = [('09:00', '10:30'),('10:30', '11:30')],range_type=['price_range','price_deviation_max_first_col'])),
    ('pv', de.PriceVelocity(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ('ppi', de.PricePerIncrement(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ])
base_df = pipe2.fit(base_df).transform(base_df)

****************************************************************************************************
Shape of dataframe after PriceDayRangeHourWise is (1105219, 8)
****************************************************************************************************
PriceVelocity : open
PriceVelocity : high
PriceVelocity : last_tick_close_10
PriceVelocity : low
PriceVelocity : last_tick_open_10
PriceVelocity : close
****************************************************************************************************
Shape of dataframe after PriceDayRangeHourWise is (1105219, 8)
****************************************************************************************************
PriceVelocity : open
PriceVelocity : high
PriceVelocity : last_tick_close_10
PriceVelocity : low
PriceVelocity : last_tick_open_10
PriceVelocity : close
****************************************************************************************************
PricePerIncrement : open
PricePerIncrement : price_velocity_hig

In [5]:
pipe3 = Pipeline([
    ('pltbc', de.PriceLastTickBreachCount(column_pattern=['close','open','momentum','volatile','transform','pattern','overlap'],columns=[],create_new_col = True,last_ticks='10min',breach_type = ['morethan','max'])),
    ('pltbc1', de.PriceLastTickBreachCount(column_pattern=['close','open','momentum','volatile','transform','pattern','overlap'],columns=[],create_new_col = True,last_ticks='60min',breach_type = ['morethan','max'])),
    ('pdrhw', de.PriceDayRangeHourWise(first_col = 'high',second_col='low',hour_range = [('09:00', '10:30'),('10:30', '11:30')],range_type=['price_range','price_deviation_max_first_col'])),
    ('pv', de.PriceVelocity(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ('ppi', de.PricePerIncrement(freq='D',shift=5,shift_column=[],shift_column_pattern=['close','open','high','low','momentum','volatile','transform','pattern','overlap'],verbose=False)),
    ])
base_df = pipe3.fit(base_df).transform(base_df)

****************************************************************************************************
PriceLastTickBreachCount : morethan : price_pervelocity_price_velocity_close_D_5_D_5
PriceLastTickBreachCount : max : price_pervelocity_price_velocity_close_D_5_D_5
PriceLastTickBreachCount : morethan : open
PriceLastTickBreachCount : max : open
PriceLastTickBreachCount : morethan : last_tick_close_10
PriceLastTickBreachCount : max : last_tick_close_10
PriceLastTickBreachCount : morethan : price_velocity_close_D_5
PriceLastTickBreachCount : max : price_velocity_close_D_5
PriceLastTickBreachCount : morethan : price_pervelocity_open_D_5
PriceLastTickBreachCount : max : price_pervelocity_open_D_5
PriceLastTickBreachCount : morethan : price_pervelocity_last_tick_open_10_D_5
PriceLastTickBreachCount : max : price_pervelocity_last_tick_open_10_D_5
PriceLastTickBreachCount : morethan : last_tick_open_10
PriceLastTickBreachCount : max : last_tick_open_10
PriceLastTickBreachCount : morethan : pr

KeyboardInterrupt: 

In [12]:
a = ['momentum_ADX', 'momentum_ADXR', 'momentum_APO', 'momentum_AROON', 'momentum_AROONOSC', 'momentum_BOP', 'momentum_CCI', 'momentum_CMO', 'momentum_DX', 'momentum_MACD', 'momentum_MACDEXT', 'momentum_MACDFIX', 'momentum_MFI', 'momentum_MINUS_DM', 'momentum_MOM', 'momentum_PLUS_DI', 'momentum_PLUS_DM', 'momentum_PPO', 'momentum_ROC', 'momentum_ROCP', 'momentum_ROCR', 'momentum_ROCR100', 'momentum_RSI', 'momentum_STOCH', 'momentum_STOCHF', 'momentum_STOCHRSI', 'momentum_TRIX', 'momentum_ULTOSC', 'momentum_WILLR', 'volumn_AD', 'volumn_ADOSC', 'volumn_OBV', 'volatile_ATR', 'volatile_NATR', 'volatile_TRANGE', 'transform_AVGPRICE', 'transform_MEDPRICE', 'transform_TYPPRICE', 'transform_WCLPRICE', 'cycle_HT_DCPERIOD', 'cycle_HT_DCPHASE', 'cycle_HT_PHASOR', 'cycle_HT_SINE', 'cycle_HT_TRENDMODE', 'pattern_2_crows', 'pattern_3_black_crows', 'pattern_3_inside_updown', 'pattern_3_line_strike', 'pattern_3_outside_updown', 'pattern_3_stars_south', 'pattern_3_adv_white_soldier', 'pattern_abondoned_baby', 'pattern_advance_block', 'pattern_belt_hold', 'pattern_breakaway', 'pattern_closing_marubozu', 'pattern_concealing_baby_swallow', 'pattern_counter_attack', 'pattern_dark_cloud_cover', 'pattern_doji', 'pattern_doji_star', 'pattern_dragonfly_doji', 'pattern_engulfing_pattern', 'pattern_evening_doji_star', 'pattern_evening_star', 'pattern_updown_gapside_white_lines', 'pattern_gravestone_doji', 'pattern_hammer', 'pattern_hanging_man', 'pattern_harami', 'pattern_harami_cross', 'pattern_high_wave_candle', 'pattern_hikkake', 'pattern_modified_hikkake', 'pattern_homing_pigeon', 'pattern_identical_3_cross', 'pattern_in_neck', 'pattern_inverted_hammer', 'pattern_kicking', 'pattern_kicking_bullbear_morubozu', 'pattern_ladder_bottom', 'pattern_long_leg_doji', 'pattern_long_line_candle', 'pattern_morubozu', 'pattern_matching_low', 'pattern_mat_holding', 'pattern_morning_doji_star', 'pattern_morning_star', 'pattern_on_neck', 'pattern_piercing', 'pattern_rickshaw_man', 'pattern_risefall_3_methods', 'pattern_separating_lines', 'pattern_shooting_star', 'pattern_short_line', 'pattern_spinning_top', 'pattern_stalled', 'pattern_stick_sandwich', 'pattern_takuri', 'pattern_tasuki_gap', 'pattern_thrusting_pattern', 'pattern_tristar_pattern', 'pattern_unique_3_river', 'pattern_upside_gap_2_crows', 'pattern_updown_gap_3_method', 'stats_beta', 'stats_pearson_coeff', 'stats_linear_reg', 'stats_linear_reg_angle', 'stats_linear_reg_intercept', 'stats_linear_reg_slope', 'stats_time_series_forecast', 'stats_variance', 'math_ACOS', 'math_ASIN', 'math_ATAN', 'math_CEIL', 'math_COS', 'math_COSH', 'math_EXP', 'math_FLOOR', 'math_LN', 'math_LOG10', 'math_SIN', 'math_SINH', 'math_SQRT', 'math_TAN', 'math_TANH', 'overlap_bolliner_bands', 'overlap_double_exp_moving_avg', 'overlap_exp_moving_avg', 'overlap_hilbert_transform', 'overlap_kaufman_adaptive_moving_avg', 'overlap_moving_avg', 'overlap_mesa_adaptive_moving_average', 'overlap_moving_avg_with_variable', 'overlap_midpoint', 'overlap_midpoint_price', 'overlap_parabolic_sar', 'overlap_parabolic_sar_ext', 'overlap_simple_moving_avg', 'overlap_triple_exp_moving_average_t3', 'overlap_triangular_moving_average', 'overlap_weighted_moving_average']
method_type = ['transform_','cycle_','pattern_','stats_','math_','overlap_']


t = [col for col in a for c in method_type if c in col]
print(t)

['transform_AVGPRICE', 'transform_MEDPRICE', 'transform_TYPPRICE', 'transform_WCLPRICE', 'cycle_HT_DCPERIOD', 'cycle_HT_DCPHASE', 'cycle_HT_PHASOR', 'cycle_HT_SINE', 'cycle_HT_TRENDMODE', 'pattern_2_crows', 'pattern_3_black_crows', 'pattern_3_inside_updown', 'pattern_3_line_strike', 'pattern_3_outside_updown', 'pattern_3_stars_south', 'pattern_3_adv_white_soldier', 'pattern_abondoned_baby', 'pattern_advance_block', 'pattern_belt_hold', 'pattern_breakaway', 'pattern_closing_marubozu', 'pattern_concealing_baby_swallow', 'pattern_counter_attack', 'pattern_dark_cloud_cover', 'pattern_doji', 'pattern_doji_star', 'pattern_dragonfly_doji', 'pattern_engulfing_pattern', 'pattern_evening_doji_star', 'pattern_evening_star', 'pattern_updown_gapside_white_lines', 'pattern_gravestone_doji', 'pattern_hammer', 'pattern_hanging_man', 'pattern_harami', 'pattern_harami_cross', 'pattern_high_wave_candle', 'pattern_hikkake', 'pattern_modified_hikkake', 'pattern_homing_pigeon', 'pattern_identical_3_cross', 

In [13]:
base_df['test'] = base_df['close'].rolling('10min', min_periods=1).apply(lambda x: (x[-1] > x[:-1]).mean()).astype(float)

In [43]:
t = base_df[0:20]
import numpy as np

In [66]:
t['test'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x[-1] < x[:-1]).sum()).fillna(0)
t['test1'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].mean()).fillna(0)
t['test2'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x > x[:].mean()).sum()).fillna(0)
t['test3'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x < x[:].mean()).sum()).fillna(0)
t['test4'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x < x[:].quantile(0.25)).sum()).fillna(0)
t['test5'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].quantile(0.25)).fillna(0)
t[['close','test','test1','test2','test3','test4','test5']]

<ipython-input-66-8723029f915a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['test'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: (x[-1] < x[:-1]).sum()).fillna(0)
<ipython-input-66-8723029f915a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['test1'] = t['close'].rolling('10min', min_periods=0).apply(lambda x: x[:].mean()).fillna(0)
<ipython-input-66-8723029f915a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,close,test,test1,test2,test3,test4,test5
2010-03-02 09:01:00,8839.80,0.0,8839.800000,0.0,0.0,0.0,8839.8000
2010-03-02 09:02:00,8852.30,0.0,8846.050000,1.0,1.0,1.0,8842.9250
2010-03-02 09:03:00,8846.65,1.0,8846.250000,2.0,1.0,1.0,8843.2250
2010-03-02 09:04:00,8846.70,1.0,8846.362500,3.0,1.0,1.0,8844.9375
2010-03-02 09:05:00,8841.60,3.0,8845.410000,3.0,2.0,1.0,8841.6000
2010-03-02 09:06:00,8836.90,5.0,8843.991667,3.0,3.0,2.0,8840.2500
2010-03-02 09:07:00,8846.35,3.0,8844.328571,4.0,3.0,2.0,8840.7000
2010-03-02 09:08:00,8855.70,0.0,8845.750000,5.0,3.0,2.0,8841.1500
2010-03-02 09:09:00,8859.15,0.0,8847.238889,3.0,6.0,2.0,8841.6000
2010-03-02 09:10:00,8864.95,0.0,8849.010000,4.0,6.0,3.0,8842.7875


In [5]:
base_df

,open,high,low,close,label_-15_1min_close,label_-30_1min_close,momentum_ADX,momentum_ADXR,momentum_APO,momentum_BOP,...,overlap_triangular_moving_average,overlap_weighted_moving_average,last_tick_close_10,last_tick_open_10,last_tick_mean_close_10min,last_tick_mean_open_10min,range_1030,range_1130,price_velocity_close_1min_15,price_perincrement_close_1min_15
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60,60.872509,46.563122,25.209295,0.429285,...,8883.028542,8891.587849,0,0,0,0,NaN,NaN,NaN,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40,60.872509,46.563122,25.209295,0.547393,...,8883.028542,8891.587849,0,0,1,1,NaN,NaN,NaN,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60,60.872509,46.563122,25.209295,-0.819005,...,8883.028542,8891.587849,1,0,2,1,NaN,NaN,NaN,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40,60.872509,46.563122,25.209295,0.410072,...,8883.028542,8891.587849,1,1,3,2,NaN,NaN,NaN,NaN
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60,60.872509,46.563122,25.209295,-0.554913,...,8883.028542,8891.587849,3,1,3,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,unknown,unknown,11.763497,15.972811,-2.321795,0.293814,...,38473.485417,38471.749254,6,8,4,5,NaN,NaN,-0.593,-0.023
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,unknown,unknown,11.976822,16.479083,-4.639423,-0.532328,...,38474.163333,38471.008179,10,4,6,5,NaN,NaN,-1.480,-0.058
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,unknown,unknown,12.011164,16.867322,-5.979167,0.100610,...,38474.471250,38470.770652,7,8,6,4,NaN,NaN,-1.950,-0.076
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,unknown,unknown,12.043054,16.846964,-5.876923,0.000000,...,38474.326458,38470.447427,6,5,6,4,NaN,NaN,-1.937,-0.076


In [10]:
for col in base_df.columns.tolist():
    print(col,len(base_df[col].isnull()),base_df[col].isna().sum())
#base_df.head()

open 1105219 0
high 1105219 0
low 1105219 0
close 1105219 0
label_-15_1min_close 1105219 0
label_-30_1min_close 1105219 0
momentum_ADX 1105219 0
momentum_ADXR 1105219 0
momentum_APO 1105219 0
momentum_BOP 1105219 0
momentum_CCI 1105219 0
momentum_CMO 1105219 0
momentum_DX 1105219 0
momentum_MACD 1105219 0
momentum_MACD_SIGNAL 1105219 0
momentum_MACD_HIST 1105219 0
momentum_MACDEXT 1105219 0
momentum_MACDEXT_SIGNAL 1105219 0
momentum_MACDEXT_HIST 1105219 0
momentum_MACDFIX 1105219 0
momentum_MACDFIX_SIGNAL 1105219 0
momentum_MACDFIX_HIST 1105219 0
momentum_MINUS_DM 1105219 0
momentum_MOM 1105219 0
momentum_PLUS_DI 1105219 0
momentum_PLUS_DM 1105219 0
momentum_PPO 1105219 0
momentum_ROC 1105219 0
momentum_ROCP 1105219 0
momentum_ROCR 1105219 0
momentum_ROCR100 1105219 0
momentum_RSI 1105219 0
momentum_SLOWK 1105219 0
momentum_SLOWD 1105219 0
momentum_FASTK 1105219 0
momentum_FASTD 1105219 0
momentum_STOCHRSI_FASTK 1105219 0
momentum_STOCHRSI_FASTD 1105219 0
momentum_TRIX 1105219 0
moment

In [9]:
base_df.dtypes

open                                 float64
high                                 float64
low                                  float64
close                                float64
label_-15_1min_close                  object
                                      ...   
overlap_midpoint                     float64
overlap_midpoint_price               float64
overlap_simple_moving_avg            float64
overlap_triangular_moving_average    float64
overlap_weighted_moving_average      float64
Length: 141, dtype: object

In [6]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('nd', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=True,impute_type = 'mean_median_imputer',convert_to_floats = True,arbitrary_impute_variable=99)),
    ('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),
    ('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),
    ('tech_indicator', de.TechnicalIndicator()),
    ('nd2', de.NormalizeDataset(columns = ['close','open','high','low'],impute_values=False,convert_to_floats = False,drop_na_rows=True)),
    ('ltgvc', de.LastTicksGreaterValuesCount(columns=['close','open'])),
   # ('pltbc', de.PriceLastTickBreachCount(columns=['close','open'])),
   # ('pdrhw', de.PriceDayRangeHourWise()),
   # ('pv', de.PriceVelocity()),
   # ('ppi', de.PricePerIncrement()),
    ])
base_df = pipe.fit(base_df).transform(base_df)


Function momentum_AROON was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_AROONOSC was unable to run, Error is Too many price arguments: expected 2 (high, low)
Function momentum_MFI was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_AD was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_ADOSC was unable to run, Error is 'Signals' object has no attribute 'volume'
Function volumn_OBV was unable to run, Error is 'Signals' object has no attribute 'volume'
Function pattern_abondoned_baby was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_dark_cloud_cover was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_doji_star was unable to run, Error is Invalid parameter value for penetration (expected float, got int)
Function pattern_morning_star was unable to r

KeyboardInterrupt: 

In [8]:
base_df.shape

(0, 143)

In [8]:
type(base_df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [9]:
obj = EqualWidthDiscretiser(variables=['open'], bins=10, return_object=False, return_boundaries=False)
obj.fit(base_df.fillna(0)).transform(base_df)

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,1,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,1,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,1,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,1,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,1,8846.40,8837.75,8841.60,60to80,40to60
...,...,...,...,...,...,...
2022-01-13 15:28:00,9,38469.15,38459.45,38467.20,unknown,unknown
2022-01-13 15:29:00,9,38479.85,38456.65,38456.65,unknown,unknown
2022-01-13 15:30:00,9,38471.55,38455.15,38465.70,unknown,unknown
2022-01-13 15:31:00,9,38466.05,38466.05,38466.05,unknown,unknown


In [8]:
pipe.get_params()

{'memory': None,
 'steps': [('labelgenerator_1', LabelCreator()),
  ('labelgenerator_2', LabelCreator(shift=-30)),
  ('technical', TechnicalIndicator())],
 'verbose': False,
 'labelgenerator_1': LabelCreator(),
 'labelgenerator_2': LabelCreator(shift=-30),
 'technical': TechnicalIndicator(),
 'labelgenerator_1__freq': '1min',
 'labelgenerator_1__shift': -15,
 'labelgenerator_1__shift_column': 'close',
 'labelgenerator_2__freq': '1min',
 'labelgenerator_2__shift': -30,
 'labelgenerator_2__shift_column': 'close'}

In [9]:
pipe.set_params()

Pipeline(steps=[('labelgenerator_1', LabelCreator()),
                ('labelgenerator_2', LabelCreator(shift=-30)),
                ('technical', TechnicalIndicator())])

In [10]:
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer

In [11]:
base_df.columns

Index(['open', 'high', 'low', 'close', 'label_-15_1min_close',
       'label_-30_1min_close', 'momentum_ADX', 'momentum_ADXR', 'momentum_APO',
       'momentum_BOP',
       ...
       'overlap_double_exp_moving_avg', 'overlap_exp_moving_avg',
       'overlap_hilbert_transform', 'overlap_kaufman_adaptive_moving_avg',
       'overlap_moving_avg', 'overlap_midpoint', 'overlap_midpoint_price',
       'overlap_simple_moving_avg', 'overlap_triangular_moving_average',
       'overlap_weighted_moving_average'],
      dtype='object', length=141)

In [5]:
n = 5
base_df['close'] = (base_df['close'].rolling(n+1, min_periods=1).apply(lambda x: (x[-1] > x[:-1]).mean()).astype(int))

In [6]:
import numpy as np

In [7]:
n = 2
x = np.concatenate([[np.nan] * (n), base_df['close'].values])

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
arr = rolling_window(x, n + 1)

base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)

<ipython-input-7-402c5888b12e>:10: RuntimeWarning: invalid value encountered in greater
  base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)


In [8]:
base_df

,open,high,low,close,new
2018-04-02 09:08:00,24235.70,24235.70,24235.70,24235.70,0
2018-04-02 09:16:00,24268.00,24300.60,24243.70,24253.10,0
2018-04-02 09:17:00,24257.40,24286.10,24219.30,24221.40,2
2018-04-02 09:18:00,24222.60,24224.40,24204.00,24220.70,2
2018-04-02 09:19:00,24218.70,24247.10,24218.70,24229.40,0
...,...,...,...,...,...
2011-12-30 15:26:00,7955.80,7956.25,7954.30,7956.25,1
2011-12-30 15:27:00,7955.80,7959.90,7955.80,7959.80,0
2011-12-30 15:28:00,7960.90,7963.45,7960.90,7961.40,0
2011-12-30 15:29:00,7959.45,7959.45,7953.30,7954.70,2


In [31]:
base_df

,open,high,low,close,label_-15_1min_close,label_-30_1min_close,new
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60,1
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40,1
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60,2
...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,unknown,unknown,1
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,unknown,unknown,2
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,unknown,unknown,1
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,unknown,unknown,0


In [34]:
last_ticks = 2
base_df = base_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["close"], how="all")
base_df['new1'] = (base_df['close'].rolling(last_ticks+1, min_periods=1)
                        .apply(lambda x: (x[-1] > x[:-1]).mean())
                        .astype(int))
base_df

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [25]:
import numpy as np
df = pd.DataFrame({"a": np.random.randint(1, high=10, size=50)})
#print(df)
def f(s):
    #d = s.to_frame().groupby(pd.cut(s, np.arange(s.min(), s.max()*1.01, 1), include_lowest=True)).transform("count")
    tmp, bins = pd.cut(
                x=s, bins=3, retbins=True, duplicates="drop"
            )
    bins = list(bins)
    bins[0] = float("-inf")
    bins[len(bins) - 1] = float("inf")
    return pd.cut(x=s, bins=bins, labels=False)

df["a"].rolling(10).apply(f)

TypeError: cannot convert the series to <class 'float'>

In [21]:
tmp, bins = pd.cut(
                x=df['a'], bins=3, retbins=True, duplicates="drop"
            )
bins

array([0.992     , 3.66666667, 6.33333333, 9.        ])

In [22]:
bins = list(bins)
bins[0] = float("-inf")
bins[len(bins) - 1] = float("inf")
bins

[-inf, 3.6666666666666665, 6.333333333333333, inf]

In [24]:
t = pd.cut(x=df['a'], bins=bins, labels=False)
t

0     0
1     2
2     2
3     2
4     0
5     1
6     2
7     2
8     1
9     2
10    0
11    0
12    2
13    2
14    0
15    1
16    1
17    1
18    1
19    0
20    1
21    0
22    0
23    2
24    1
25    1
26    2
27    2
28    1
29    1
30    2
31    0
32    0
33    2
34    2
35    1
36    0
37    1
38    2
39    0
40    1
41    1
42    0
43    0
44    1
45    0
46    1
47    2
48    1
49    0
Name: a, dtype: int64

In [30]:
i = pd.date_range('2018-04-09', periods=100, freq='30min')
ts = pd.DataFrame({'A': list(range(100))}, index=i)
ts

,A
2018-04-09 00:00:00,0
2018-04-09 00:30:00,1
2018-04-09 01:00:00,2
2018-04-09 01:30:00,3
2018-04-09 02:00:00,4
...,...
2018-04-10 23:30:00,95
2018-04-11 00:00:00,96
2018-04-11 00:30:00,97
2018-04-11 01:00:00,98


In [32]:
ts.between_time('09:00', '10:30')['A']

2018-04-09 09:00:00    18
2018-04-09 09:30:00    19
2018-04-09 10:00:00    20
2018-04-09 10:30:00    21
2018-04-10 09:00:00    66
2018-04-10 09:30:00    67
2018-04-10 10:00:00    68
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [63]:
from datetime import datetime


In [83]:
s1 = ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
s1.index = s1.index + pd.DateOffset(minutes=60*10.5)
s1

2018-04-09 10:30:00    21
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [71]:
s1.index

DatetimeIndex(['2018-04-09', '2018-04-10'], dtype='datetime64[ns]', freq='D')

In [42]:
s1.dtypes

A    int64
dtype: object

In [84]:
merge=pd.merge(ts,s1, how='outer', left_index=True, right_index=True)
merge = merge.fillna(method='ffill')
#merge

In [85]:
merge.between_time('09:00', '12:30')

,A_x,A_y
2018-04-09 09:00:00,18,NaN
2018-04-09 09:30:00,19,NaN
2018-04-09 10:00:00,20,NaN
2018-04-09 10:30:00,21,21.0
2018-04-09 11:00:00,22,21.0
2018-04-09 11:30:00,23,21.0
2018-04-09 12:00:00,24,21.0
2018-04-09 12:30:00,25,21.0
2018-04-10 09:00:00,66,21.0
2018-04-10 09:30:00,67,21.0


In [89]:
base_df.head()

,open,high,low,close
20180402 09:08,24235.7,24235.7,24235.7,24235.7
20180402 09:16,24268.0,24300.6,24243.7,24253.1
20180402 09:17,24257.4,24286.1,24219.3,24221.4
20180402 09:18,24222.6,24224.4,24204.0,24220.7
20180402 09:19,24218.7,24247.1,24218.7,24229.4


In [92]:
base_df.index = pd.to_datetime(base_df.index)
base_df = base_df.sort_index()
base_df = base_df[~base_df.index.duplicated(keep='first')]  
s1 = base_df['high'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - base_df['low'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
#s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
c = [int(i) for i in x.split(':')]
s1.index = s1.index + pd.DateOffset(minutes=c[0]*60 + c[1])
pd.DataFrame(s1)

2010-03-02 10:30:00     82.70
2010-03-03 10:30:00    101.85
2010-03-04 10:30:00     53.90
2010-03-05 10:30:00     82.45
2010-03-06 10:30:00       NaN
                        ...  
2022-01-09 10:30:00       NaN
2022-01-10 10:30:00    332.25
2022-01-11 10:30:00    453.00
2022-01-12 10:30:00    246.85
2022-01-13 10:30:00    301.50
Freq: D, Length: 4336, dtype: float64

In [98]:
s1 = pd.DataFrame(s1,columns=['range_1030'])

In [100]:
pd.options.display.max_rows = 100

In [106]:
merge=pd.merge(base_df,s1, how='outer', left_index=True, right_index=True)
merge['range_1030'] = merge['range_1030'].fillna(method='ffill')
merge

,open,high,low,close,range_1030
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,NaN
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,NaN
...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,301.5
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,301.5
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,301.5
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,301.5


In [108]:
merge.between_time('10:29', '10:31')

,open,high,low,close,range_1030
2010-03-02 10:29:00,8864.05,8868.35,8863.60,8867.20,NaN
2010-03-02 10:30:00,8865.05,8868.80,8865.05,8868.25,82.70
2010-03-02 10:31:00,8867.70,8870.95,8867.70,8869.30,82.70
2010-03-03 10:29:00,9020.85,9020.85,9017.75,9018.55,82.70
2010-03-03 10:30:00,9020.10,9021.00,9019.05,9020.50,101.85
...,...,...,...,...,...
2022-01-12 10:30:00,38835.50,38841.40,38828.80,38840.60,246.85
2022-01-12 10:31:00,38838.20,38840.65,38818.35,38828.90,246.85
2022-01-13 10:29:00,38475.45,38478.15,38459.95,38464.00,246.85
2022-01-13 10:30:00,38461.60,38465.05,38455.30,38458.15,301.50


In [109]:
import datetime as DT
t1 = DT.datetime.strptime('00:05:36.0100000', '%H:%M:%S.%f0')
t2 = DT.datetime(1900,1,1)

print((t1-t2).total_seconds() / 60.0)

5.6001666666666665


In [122]:
x = '10:30'
c = [int(i) for i in x.split(':')]

In [124]:
c[0]*60 + c[1]

630

In [121]:
x.split(':')

['10', '30']